### FE621 - Homework #4

**Author**: Sid Bhatia

**Date**: May 2nd, 2024

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Sveinn Olafsson

**TA**: Dong Woo Kim